# COX Multiple Regression Tutorial

A comprehensive step-by-step tutorial on performing Cox Proportional Hazards Multiple Regression for survival analysis.

## 📋 Overview

This tutorial will guide you through:
1. Loading and exploring survival data
2. Data preprocessing and preparation
3. Fitting Cox Proportional Hazards models
4. Interpreting results and hazard ratios
5. Visualizing survival curves
6. Checking model assumptions

## 🔧 Setup

First, let's install the required packages.


In [ ]:
# Install required packages
!pip install -q pandas numpy lifelines matplotlib seaborn scipy scikit-learn

print("✅ All packages installed successfully!")


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter, KaplanMeierFitter
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")


## 📊 Step 1: Load and Explore the Data

### Option A: Load from GitHub (if repository is public)

If this notebook is linked to a GitHub repository, you can load data directly:


In [ ]:
# Option A: Load data from GitHub (uncomment to use):
"""
github_url = "https://raw.githubusercontent.com/aralap/3DG_Tutorials/main/data/sample_survival_data.csv"
data = pd.read_csv(github_url)
metadata = pd.read_csv("https://raw.githubusercontent.com/aralap/3DG_Tutorials/main/data/sample_metadata.csv")
"""


### Option B: Generate Sample Data

Alternatively, we can generate sample data directly in the notebook:


In [ ]:
# Option B: Generate sample data
def generate_sample_survival_data(n_samples=500, random_seed=42):
    """Generate synthetic survival data for COX regression analysis."""
    np.random.seed(random_seed)
    
    patient_ids = range(1, n_samples + 1)
    age = np.random.randint(30, 81, n_samples)
    gender = np.random.choice(['Male', 'Female'], n_samples, p=[0.55, 0.45])
    treatment = np.random.choice(['A', 'B', 'C'], n_samples, p=[0.35, 0.35, 0.30])
    biomarker1 = np.random.normal(50, 15, n_samples)
    biomarker2 = np.random.normal(100, 25, n_samples)
    
    # Generate survival times based on covariates
    baseline_hazard = 0.02
    treatment_effect = np.where(treatment == 'B', 0.5, np.where(treatment == 'A', 0.8, 1.0))
    age_effect = age / 80
    biomarker_effect = biomarker1 / 100
    
    hazard_rate = baseline_hazard * treatment_effect * (1 + age_effect * 0.5) * (1 + biomarker_effect * 0.3)
    survival_time = np.random.exponential(1 / hazard_rate)
    survival_time = np.minimum(survival_time, 1825)
    
    # Generate censoring (30% censoring rate)
    censoring_time = np.random.uniform(365, 1825, n_samples)
    event = (survival_time <= censoring_time).astype(int)
    survival_time_observed = np.where(event == 1, survival_time, censoring_time)
    
    data = pd.DataFrame({
        'patient_id': patient_ids,
        'survival_time': survival_time_observed,
        'event': event,
        'age': age,
        'gender': gender,
        'treatment': treatment,
        'biomarker1': np.round(biomarker1, 2),
        'biomarker2': np.round(biomarker2, 2)
    })
    
    return data

# Generate data
data = generate_sample_survival_data(n_samples=500, random_seed=42)
print("✅ Sample data generated successfully!")
print(f"Data shape: {data.shape}")
print(f"Events: {data['event'].sum()}/{len(data)} ({100*data['event'].sum()/len(data):.1f}%)")


In [ ]:
# Display first few rows
print("First 5 rows:")
print(data.head())
print("\n" + "="*80 + "\n")

# Display data info
print("Data information:")
print(data.info())
print("\n" + "="*80 + "\n")

# Display summary statistics
print("Summary statistics:")
print(data.describe())
print("\n" + "="*80 + "\n")

# Check for missing values
print("Missing values:")
print(data.isnull().sum())


### Visualize Data Distribution

Let's create some exploratory visualizations:


In [ ]:
# Create subplots for exploratory analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Age distribution
axes[0, 0].hist(data['age'], bins=20, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Age Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Age (years)')
axes[0, 0].set_ylabel('Frequency')

# Survival time distribution
axes[0, 1].hist(data['survival_time'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 1].set_title('Survival Time Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Time (days)')
axes[0, 1].set_ylabel('Frequency')

# Event status
event_counts = data['event'].value_counts()
axes[0, 2].bar(['Censored', 'Event'], [event_counts[0], event_counts[1]], 
               color=['orange', 'red'], edgecolor='black')
axes[0, 2].set_title('Event Status Distribution', fontweight='bold')
axes[0, 2].set_ylabel('Count')

# Gender distribution
gender_counts = data['gender'].value_counts()
axes[1, 0].bar(gender_counts.index, gender_counts.values, edgecolor='black')
axes[1, 0].set_title('Gender Distribution', fontweight='bold')
axes[1, 0].set_ylabel('Count')

# Treatment distribution
treatment_counts = data['treatment'].value_counts()
axes[1, 1].bar(treatment_counts.index, treatment_counts.values, edgecolor='black')
axes[1, 1].set_title('Treatment Distribution', fontweight='bold')
axes[1, 1].set_ylabel('Count')

# Biomarker1 distribution
axes[1, 2].hist(data['biomarker1'], bins=20, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1, 2].set_title('Biomarker1 Distribution', fontweight='bold')
axes[1, 2].set_xlabel('Biomarker1 Level')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


## 🔄 Step 2: Data Preprocessing

Before fitting the Cox model, we need to:
1. Handle missing values
2. Encode categorical variables
3. Prepare the data in the correct format


In [ ]:
# Check for missing values
print("Missing values before preprocessing:")
print(data.isnull().sum())
print("\n" + "="*80 + "\n")

# Handle missing values (if any)
data_clean = data.dropna().copy()
print(f"Data shape: {data.shape} → {data_clean.shape} after removing missing values")

# Encode categorical variables
le_gender = LabelEncoder()
le_treatment = LabelEncoder()

data_clean['gender_encoded'] = le_gender.fit_transform(data_clean['gender'])
data_clean['treatment_encoded'] = le_treatment.fit_transform(data_clean['treatment'])

print("\nCategorical encoding:")
print(f"Gender: {dict(zip(le_gender.classes_, le_gender.transform(le_gender.classes_)))}")
print(f"Treatment: {dict(zip(le_treatment.classes_, le_treatment.transform(le_treatment.classes_)))}")


### Prepare Data for Cox Regression

The Cox model requires:
- **Duration column**: Time until event or censoring
- **Event column**: Binary indicator (1 = event occurred, 0 = censored)
- **Covariates**: Predictor variables


In [ ]:
# Prepare data for Cox regression
cox_data = data_clean[['survival_time', 'event', 'age', 'gender_encoded', 
                       'treatment_encoded', 'biomarker1', 'biomarker2']].copy()

# Rename columns for clarity (lifelines uses 'duration' and 'event')
cox_data.columns = ['duration', 'event', 'age', 'gender', 
                    'treatment', 'biomarker1', 'biomarker2']

print("Data prepared for Cox regression:")
print(cox_data.head())
print(f"\nData shape: {cox_data.shape}")
print(f"Events: {cox_data['event'].sum()}/{len(cox_data)} ({100*cox_data['event'].sum()/len(cox_data):.1f}%)")


## 📈 Step 3: Fit Cox Proportional Hazards Model

Now we'll fit the Cox regression model with multiple covariates:


In [ ]:
# Initialize Cox Proportional Hazards Fitter
cph = CoxPHFitter()

# Fit the model
print("Fitting Cox Proportional Hazards model...")
cph.fit(cox_data, duration_col='duration', event_col='event')

print("\n✅ Model fitted successfully!")


### View Model Summary

The summary includes:
- **Hazard Ratios (HR)**: Effect of each covariate on survival
- **Confidence Intervals**: 95% CI for each HR
- **P-values**: Statistical significance
- **Concordance Index**: Model performance (similar to AUC)


In [ ]:
# Print detailed summary
print("="*80)
print("COX REGRESSION RESULTS")
print("="*80)
cph.print_summary()
print("="*80)


### Access Model Parameters

You can also access specific model parameters programmatically:


In [ ]:
# Get hazard ratios
print("Hazard Ratios:")
print(cph.hazard_ratios_)
print("\n" + "="*80 + "\n")

# Get confidence intervals
print("95% Confidence Intervals:")
print(cph.confidence_intervals_)
print("\n" + "="*80 + "\n")

# Get p-values
print("P-values:")
print(cph.summary['p'])
print("\n" + "="*80 + "\n")

# Get concordance index (C-index)
print(f"Concordance Index (C-index): {cph.concordance_index_:.4f}")
print("(C-index > 0.7 indicates good discrimination ability)")


## 📊 Step 4: Visualize Results

### Plot Hazard Ratios

Hazard ratios (HR) show the effect of each covariate:
- **HR = 1**: No effect
- **HR > 1**: Increased hazard (worse survival)
- **HR < 1**: Decreased hazard (better survival)


In [ ]:
# Plot hazard ratios with confidence intervals
fig, ax = plt.subplots(figsize=(10, 6))
cph.plot(ax=ax)
ax.set_title('Hazard Ratios with 95% Confidence Intervals', fontsize=14, fontweight='bold')
ax.axvline(x=1, color='red', linestyle='--', alpha=0.5, label='No effect (HR=1)')
ax.set_xlabel('Hazard Ratio', fontsize=12)
ax.set_ylabel('Covariate', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### Kaplan-Meier Survival Curves

Kaplan-Meier curves show the survival probability over time, stratified by groups:


In [ ]:
# Create Kaplan-Meier curves
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# By gender
kmf = KaplanMeierFitter()
ax1 = axes[0]
for gender in data_clean['gender'].unique():
    mask = data_clean['gender'] == gender
    kmf.fit(data_clean[mask]['survival_time'], 
            data_clean[mask]['event'], 
            label=f'{gender}')
    kmf.plot_survival_function(ax=ax1)

ax1.set_title('Kaplan-Meier Curves by Gender', fontsize=12, fontweight='bold')
ax1.set_xlabel('Time (days)', fontsize=11)
ax1.set_ylabel('Survival Probability', fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.legend()

# By treatment
kmf = KaplanMeierFitter()
ax2 = axes[1]
for treatment in sorted(data_clean['treatment'].unique()):
    mask = data_clean['treatment'] == treatment
    kmf.fit(data_clean[mask]['survival_time'], 
            data_clean[mask]['event'], 
            label=f'Treatment {treatment}')
    kmf.plot_survival_function(ax=ax2)

ax2.set_title('Kaplan-Meier Curves by Treatment', fontsize=12, fontweight='bold')
ax2.set_xlabel('Time (days)', fontsize=11)
ax2.set_ylabel('Survival Probability', fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()


## ✅ Step 5: Check Model Assumptions

### Proportional Hazards Assumption

The Cox model assumes that hazard ratios are constant over time. We can check this using Schoenfeld residuals:


In [ ]:
# Check proportional hazards assumption
print("Checking proportional hazards assumption...")
print("="*80)

try:
    cph.check_assumptions(cox_data, p_value_threshold=0.05, show_plots=True)
    print("\n✅ Proportional hazards assumption checked.")
except Exception as e:
    print(f"\n⚠️  Note: {e}")
    print("This is normal if the test requires specific conditions.")
    print("Visual inspection of Schoenfeld residuals can still be performed.")


## 📝 Step 6: Interpreting Results

### How to Interpret Hazard Ratios

1. **HR = 1.0**: The covariate has no effect on survival
2. **HR > 1.0**: Higher values of the covariate are associated with **worse** survival (increased hazard)
3. **HR < 1.0**: Higher values of the covariate are associated with **better** survival (decreased hazard)

### Example Interpretation

If `treatment = 1` (Treatment B) has HR = 0.65 with 95% CI [0.45, 0.94] and p < 0.05:
- Treatment B reduces the hazard of the event by **35%** compared to the reference group
- The effect is statistically significant
- The 95% CI does not include 1.0, confirming significance

### Model Performance

- **Concordance Index (C-index)**: Similar to AUC in logistic regression
  - 0.5 = Random guessing
  - 0.7 = Good discrimination
  - 1.0 = Perfect discrimination

### Summary Statistics

- **Log-likelihood**: Measure of model fit (higher is better)
- **AIC (Akaike Information Criterion)**: Model selection metric (lower is better)


## 🎯 Additional Analyses

### Predict Individual Survival Probabilities

You can use the fitted model to predict survival probabilities for new patients:


In [ ]:
# Example: Predict survival probability for a new patient
new_patient = pd.DataFrame({
    'age': [65],
    'gender': [1],  # Encoded: 1 = Female, 0 = Male
    'treatment': [1],  # Encoded: 1 = Treatment B, 0 = Treatment A, 2 = Treatment C
    'biomarker1': [55.0],
    'biomarker2': [110.0]
})

# Predict survival probability at specific time points
time_points = [365, 730, 1095, 1460]  # 1, 2, 3, 4 years
survival_probs = cph.predict_survival_function(new_patient, times=time_points)

print("Survival Probabilities for New Patient:")
print(f"Age: {new_patient['age'].iloc[0]}, Gender: Female, Treatment: B")
print(f"Biomarker1: {new_patient['biomarker1'].iloc[0]}, Biomarker2: {new_patient['biomarker2'].iloc[0]}")
print("\nTime (days) | Survival Probability")
print("-" * 40)
for t, prob in zip(time_points, survival_probs.iloc[0]):
    print(f"{t:>12} | {prob:.4f}")


## 📚 Summary

In this tutorial, we learned how to:

1. ✅ Load and explore survival data
2. ✅ Preprocess data for Cox regression
3. ✅ Fit Cox Proportional Hazards models with multiple covariates
4. ✅ Interpret hazard ratios and model results
5. ✅ Visualize results with Kaplan-Meier curves
6. ✅ Check proportional hazards assumption
7. ✅ Make predictions for new patients

## 🔗 Next Steps

- Try different combinations of covariates
- Explore interaction terms (e.g., treatment × age)
- Consider stratified models if proportional hazards are violated
- Compare models using AIC or likelihood ratio tests

## 📖 Resources

- [Lifelines Documentation](https://lifelines.readthedocs.io/)
- [Survival Analysis Guide](https://www.statsdirect.com/help/survival_analysis/cox_regression.htm)
- Cox, D. R. (1972). Regression models and life-tables. *Journal of the Royal Statistical Society: Series B*, 34(2), 187-202.

---

**Happy Analyzing! 📊**
